In [1]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from face_segmenter import segment_image
import numpy as np
import cv2
from joblib import load

In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]).astype('uint8')

In [3]:
pipe = Pipeline([('pca', PCA(0.99)), ('kmeans', KMeans(64))])

In [4]:
img = cv2.imread("images/dataset-cover.png")

In [5]:
img = rgb2gray(np.array(img,dtype='uint8'))

In [6]:
img.shape

(379, 1800)

In [7]:
clf = load('pipe_pca099_kmeans64.joblib')

In [8]:
def next_pyramid_step(img,scale = 0.9):
    return cv2.resize(img,None,fy=scale,fx=scale)

In [9]:
img_pyramid = [img]
for i in range(10):
    img = next_pyramid_step(img,0.9)
    img_pyramid.append(img)

In [10]:
P_pos_q_cara = np.load("P_pos_q_cara.npy")
P_q_nocara = np.load("P_q_nocara.npy")
P_q_cara = np.load("P_q_cara.npy")

P_norm_pos_q_cara = P_pos_q_cara / P_pos_q_cara.sum(axis=0)
P_norm_q_cara = P_q_cara / P_q_cara.sum()
P_norm_q_nocara = P_q_nocara / P_q_nocara.sum()

In [19]:
def is_cara_region(region:np.ndarray)-> bool:
    region = region.flatten()
    den = np.prod(P_norm_q_nocara[region]/ (16*64)) 
    num = np.prod(P_norm_pos_q_cara[np.arange(0,16,1,dtype=int),region]) * np.prod(P_norm_q_cara[region])

    res = num / den
    _lambda = 0.0001
    
    return res > _lambda

In [13]:
def find_caras_in_region(classes_matrix:np.ndarray,square_side = 4)-> list[tuple[int]]:
    """ 
    Encuentra las regiones que son caras y devuelve una 
    lista de los que encuentra, señalando la esquina superior izquierda
    """
    height,width = classes_matrix.shape
    height -= square_side
    width -= square_side
    caras = []
    for i,j in np.ndindex((height + 1,width +1)):
        if is_cara_region(classes_matrix[i:i+square_side,j:j+square_side]):
            caras.append((i,j))
    return caras

In [15]:
def get_real_square(caras:list[tuple[int]],scale_factor:int,padding:tuple[int,int]):
    res = []
    h,w = padding
    for i,j in caras:
        top = scale_factor * (h + 4 * i)
        left = scale_factor * (w + 4 * j)
        right = scale_factor * (w * 4 * (j + 4))
        bottom = scale_factor * (h * 4 * (i + 4))

        res.append({"top":top,
                    "left":left,
                    "right":right,
                    "bottom":bottom})
    return res

In [20]:
stride = 2
squares = []
for h in range(4):
    for w in range(4):
        mini_patches,shape = segment_image(img,8,margin_top=h*stride,margin_left=w*stride)
        mini_patches = mini_patches.reshape(-1,8*8)
        y_pred = clf.predict(mini_patches)
        y_pred = y_pred.reshape(shape[0]//8,shape[1]//8)
        # Recuperar las reguiones con caras y guardar la lista de cuadrados
        caras = find_caras_in_region(y_pred)
        squares += get_real_square(caras,scale_factor=1,padding=(h,w))


/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  res = num / den
/tmp/ipykernel_10198/3178668715.py:6: RuntimeWarning: divide by zero enc

In [22]:
len(squares)

15249

In [13]:
img_tile = np.lib.stride_tricks.as_strided(img,shape=(img.shape[0]- 32,img.shape[1]-32,32,32),strides=img.strides + img.strides, writeable=False).reshape(-1,32,32)